In [ ]:
from pathlib import Path
import requests
import uuid
import json
import base64

from context_tracer.trace_implementations.trace_server.trace_server import running_server, SpanPayload

In [ ]:
db_path = Path("./") / "export" / "test.sqlite"

span_id=SpanPayload.id_to_str(uuid.uuid1().bytes)

span = SpanPayload.from_bytes_ids(
    name="test",
    data_json=json.dumps({"test": "test"}),
    parent_id=None,
)
print(f"{span=}")
print(f"{span.model_dump_byte_ids()=}")
print()

with running_server(db_path=db_path) as server:
    url = f"http://localhost:{server.port}"
    # Make a request to the server
    resp = requests.get(f"{url}/api/status/ready")
    assert resp.status_code == 200
    # Send a span
    resp = requests.put(f"{url}/api/span/{span_id}", json=span.model_dump())
    print(f"{resp.status_code=};  {resp=};  {resp.text=}")
    print(f"{resp.request.body=}")
    assert resp.status_code == 200
    # Get the span
    resp = requests.get(f"{url}/api/span/{span_id}")
    print(f"{resp.status_code=};  {resp=};  {resp.text=}")
    assert resp.status_code == 200
    same_span = SpanPayload.model_validate_json(resp.text)
    print(f"{same_span=}")
    assert same_span == span

In [ ]:
resp.request.body